In [145]:
import requests
import pandas as pd
from collections import defaultdict
from lxml import html
import numpy as np
import re
from urllib.parse import unquote
import time
from pprint import pprint

In [146]:
catalog = pd.read_csv('collection_catalog_full.csv', sep=';')

In [147]:
catalog

,author_surname,fbusta_id,author_id,author_name,book_id,book_title,polka_title,size,year,wiki_book,wiki_author
0,Абрамов,158030,0,Фёдор Александрович,1,Братья и сестры,NaN,247,1958.0,https://ru.wikipedia.org/wiki/%D0%91%D1%80%D0%...,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
1,Абрамов,158033,0,Фёдор Александрович,2,Две зимы и три лета,NaN,293,1968.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
2,Абрамов,158036,0,Фёдор Александрович,3,Пути-перепутья,NaN,254,1973.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
3,Абрамов,158032,0,Фёдор Александрович,4,Дом,NaN,242,1978.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%...
4,Аверченко,159215,9,Аркадий Тимофеевич,40,Дюжина ножей в спину революции,NaN,39,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
5,Аверченко,169499,9,Аркадий Тимофеевич,41,О маленьких – для больших,NaN,123,NaN,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
6,Аверченко,159231,9,Аркадий Тимофеевич,42,Подходцев и двое других,NaN,128,1917.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
7,Аверченко,417321,9,Аркадий Тимофеевич,43,Разумная экономия,NaN,5,1912.0,NaN,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
8,Аверченко,159241,9,Аркадий Тимофеевич,44,Шутка мецената,NaN,105,1925.0,https://ru.wikipedia.org/wiki/%D0%A8%D1%83%D1%...,https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D0%...
9,Айтматов,150859,10,Чингиз,45,Белый пароход,NaN,113,1970.0,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D0%...,https://ru.wikipedia.org/wiki/%D0%90%D0%B9%D1%...


In [148]:
def my_unquote(link):
    if link is np.nan:
        return link
    else:
        return unquote(link).split('#')[0]

In [149]:
def href_normalize(link):
    return re.findall(r'/wiki/.+', link)[0].split('#')[0]

In [150]:
def process_links(links):
    res = defaultdict(set)
    all_hrefs = defaultdict(set)
    hrefs_normalized = set(map(href_normalize, links))
    for link in links:
        response = requests.get(link)
        assert response.status_code == 200
        dom = html.fromstring(response.text)
        for href in dom.xpath('//a/@href'):
            if 'wiki' not in href:
                continue
            norm_href = unquote(href).split('#')[0]
            all_hrefs[link].add(norm_href)
            if norm_href in hrefs_normalized and norm_href not in link:
                res[link].add(norm_href)
        time.sleep(1)
    return res, all_hrefs

In [151]:
catalog['wiki_book'] = catalog['wiki_book'].apply(my_unquote)
catalog['wiki_author'] = catalog['wiki_author'].apply(my_unquote)

In [119]:
catalog

,author_surname,fbusta_id,author_id,author_name,book_id,book_title,polka_title,size,year,wiki_book,wiki_author
0,Абрамов,158030.0,0.0,Фёдор Александрович,1.0,Братья и сестры,NaN,247.0,1958.0,https://ru.wikipedia.org/wiki/Братья_и_сёстры_...,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А..."
1,Абрамов,158033.0,0.0,Фёдор Александрович,2.0,Две зимы и три лета,NaN,293.0,1968.0,NaN,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А..."
2,Абрамов,158036.0,0.0,Фёдор Александрович,3.0,Пути-перепутья,NaN,254.0,1973.0,NaN,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А..."
3,Абрамов,158032.0,0.0,Фёдор Александрович,4.0,Дом,NaN,242.0,1978.0,NaN,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А..."
4,Аверченко,159215.0,9.0,Аркадий Тимофеевич,40.0,Дюжина ножей в спину революции,NaN,39.0,NaN,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад..."
5,Аверченко,169499.0,9.0,Аркадий Тимофеевич,41.0,О маленьких – для больших,NaN,123.0,NaN,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад..."
6,Аверченко,159231.0,9.0,Аркадий Тимофеевич,42.0,Подходцев и двое других,NaN,128.0,1917.0,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад..."
7,Аверченко,417321.0,9.0,Аркадий Тимофеевич,43.0,Разумная экономия,NaN,5.0,1912.0,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад..."
8,Аверченко,159241.0,9.0,Аркадий Тимофеевич,44.0,Шутка мецената,NaN,105.0,1925.0,https://ru.wikipedia.org/wiki/Шутка_Мецената,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад..."
9,Айтматов,150859.0,10.0,Чингиз,45.0,Белый пароход,NaN,113.0,1970.0,https://ru.wikipedia.org/wiki/Белый_пароход,"https://ru.wikipedia.org/wiki/Айтматов,_Чингиз..."


In [152]:
catalog.to_csv('collection_catalog_full.csv', sep=';')

In [153]:
links_authors = set(map(my_unquote, catalog['wiki_author'].dropna()))

In [154]:
links_authors

{'https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_Александрович',
 'https://ru.wikipedia.org/wiki/Аверченко,_Аркадий_Тимофеевич',
 'https://ru.wikipedia.org/wiki/Айтматов,_Чингиз_Торекулович',
 'https://ru.wikipedia.org/wiki/Алексиевич,_Светлана_Александровна',
 'https://ru.wikipedia.org/wiki/Алексин,_Анатолий_Георгиевич',
 'https://ru.wikipedia.org/wiki/Алешковский,_Юз',
 'https://ru.wikipedia.org/wiki/Андреев,_Леонид_Николаевич',
 'https://ru.wikipedia.org/wiki/Антоний_Погорельский',
 'https://ru.wikipedia.org/wiki/Бажов,_Павел_Петрович',
 'https://ru.wikipedia.org/wiki/Белов,_Василий_Иванович',
 'https://ru.wikipedia.org/wiki/Беляев,_Александр_Романович',
 'https://ru.wikipedia.org/wiki/Бестужев,_Александр_Александрович',
 'https://ru.wikipedia.org/wiki/Битов,_Андрей_Георгиевич',
 'https://ru.wikipedia.org/wiki/Борис_Акунин',
 'https://ru.wikipedia.org/wiki/Братья_Вайнеры',
 'https://ru.wikipedia.org/wiki/Братья_Стругацкие',
 'https://ru.wikipedia.org/wiki/Булгаков,_Михаил_Афанасьевич'

In [155]:
hrefs, all_hrefs = process_links(links_authors)

In [156]:
links_books = set(map(my_unquote, catalog['wiki_book'].dropna()))

In [157]:
links_books

{'https://ru.wikipedia.org/wiki/А_зори_здесь_тихие_(повесть)',
 'https://ru.wikipedia.org/wiki/Алтын-толобас',
 'https://ru.wikipedia.org/wiki/Алые_паруса',
 'https://ru.wikipedia.org/wiki/Антоновские_яблоки',
 'https://ru.wikipedia.org/wiki/Архипелаг_ГУЛАГ',
 'https://ru.wikipedia.org/wiki/Аэлита_(роман)',
 'https://ru.wikipedia.org/wiki/Басурман_(роман)',
 'https://ru.wikipedia.org/wiki/Безумная_Евдокия',
 'https://ru.wikipedia.org/wiki/Белая_гвардия_(роман)',
 'https://ru.wikipedia.org/wiki/Белая_голубка_Кордовы',
 'https://ru.wikipedia.org/wiki/Белеет_парус_одинокий',
 'https://ru.wikipedia.org/wiki/Белые_одежды_(роман)',
 'https://ru.wikipedia.org/wiki/Белый_Бим_Чёрное_ухо_(повесть)',
 'https://ru.wikipedia.org/wiki/Белый_пароход',
 'https://ru.wikipedia.org/wiki/Бесы_(роман)',
 'https://ru.wikipedia.org/wiki/Блистающий_мир',
 'https://ru.wikipedia.org/wiki/Братья_Карамазовы',
 'https://ru.wikipedia.org/wiki/Братья_и_сёстры_(роман)',
 'https://ru.wikipedia.org/wiki/Бронзовая_птица

In [158]:
hrefs_books, all_hrefs_books = process_links(links_books)

In [159]:
hrefs_books

defaultdict(set,
            {'https://ru.wikipedia.org/wiki/Мать_(роман)': {'/wiki/Дело_Артамоновых_(роман)'},
             'https://ru.wikipedia.org/wiki/Соборяне': {'/wiki/Леди_Макбет_Мценского_уезда',
              '/wiki/Очарованный_странник'},
             'https://ru.wikipedia.org/wiki/Рассказ_о_семи_повешенных': {'/wiki/Жизнь_Василия_Фивейского',
              '/wiki/Иуда_Искариот_(повесть)',
              '/wiki/Красный_смех'},
             'https://ru.wikipedia.org/wiki/В_круге_первом': {'/wiki/Архипелаг_ГУЛАГ',
              '/wiki/Матрёнин_двор',
              '/wiki/Один_день_Ивана_Денисовича',
              '/wiki/Раковый_корпус'},
             'https://ru.wikipedia.org/wiki/Архипелаг_ГУЛАГ': {'/wiki/В_круге_первом',
              '/wiki/Матрёнин_двор',
              '/wiki/Один_день_Ивана_Денисовича',
              '/wiki/Раковый_корпус'},
             'https://ru.wikipedia.org/wiki/Пикник_на_обочине': {'/wiki/Трудно_быть_богом',
              '/wiki/Улитка_на_склоне'},


In [160]:
catalog_authors = catalog[['author_id', 'wiki_author']].drop_duplicates().dropna()
base = 'https://ru.wikipedia.org'
hrefs_ids = defaultdict(set)
for key_a in hrefs:
    key_a_id = catalog_authors[catalog_authors['wiki_author'] == key_a]['author_id'].item()
    close_a = hrefs[key_a]
#     print(close_a)
    for c_a in close_a:
#         print()
#         print(catalog_authors[catalog_authors['wiki_author'] == base + c_a])
        c_a_id = catalog_authors[catalog_authors['wiki_author'] == base + c_a]['author_id'].item()
#         print(c_a_id)
        hrefs_ids[key_a_id].add(c_a_id)
        hrefs_ids[c_a_id].add(key_a_id)
    

In [185]:
catalog['author_com_links'] = np.nan
catalog['author_com_links'] = catalog['author_com_links'].astype(object)

In [186]:
for k in hrefs_ids:
    for i in catalog[catalog['author_id'] == k].index:
        catalog.at[i, 'author_com_links'] = '_'.join([str(j) for j in hrefs_ids[k]])

In [187]:
catalog

,author_surname,fbusta_id,author_id,author_name,book_id,book_title,polka_title,size,year,wiki_book,wiki_author,author_com_links
0,Абрамов,158030,0,Фёдор Александрович,1,Братья и сестры,NaN,247,1958.0,https://ru.wikipedia.org/wiki/Братья_и_сёстры_...,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А...",22_119
1,Абрамов,158033,0,Фёдор Александрович,2,Две зимы и три лета,NaN,293,1968.0,NaN,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А...",22_119
2,Абрамов,158036,0,Фёдор Александрович,3,Пути-перепутья,NaN,254,1973.0,NaN,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А...",22_119
3,Абрамов,158032,0,Фёдор Александрович,4,Дом,NaN,242,1978.0,NaN,"https://ru.wikipedia.org/wiki/Абрамов,_Фёдор_А...",22_119
4,Аверченко,159215,9,Аркадий Тимофеевич,40,Дюжина ножей в спину революции,NaN,39,NaN,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад...",62
5,Аверченко,169499,9,Аркадий Тимофеевич,41,О маленьких – для больших,NaN,123,NaN,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад...",62
6,Аверченко,159231,9,Аркадий Тимофеевич,42,Подходцев и двое других,NaN,128,1917.0,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад...",62
7,Аверченко,417321,9,Аркадий Тимофеевич,43,Разумная экономия,NaN,5,1912.0,NaN,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад...",62
8,Аверченко,159241,9,Аркадий Тимофеевич,44,Шутка мецената,NaN,105,1925.0,https://ru.wikipedia.org/wiki/Шутка_Мецената,"https://ru.wikipedia.org/wiki/Аверченко,_Аркад...",62
9,Айтматов,150859,10,Чингиз,45,Белый пароход,NaN,113,1970.0,https://ru.wikipedia.org/wiki/Белый_пароход,"https://ru.wikipedia.org/wiki/Айтматов,_Чингиз...",NaN


In [193]:
catalog_books = catalog[['book_id', 'wiki_book']].drop_duplicates().dropna()
base = 'https://ru.wikipedia.org'
hrefs_ids_books = defaultdict(set)
for key_a in hrefs_books:
    key_a_id = catalog_books[catalog_books['wiki_book'] == key_a]['book_id'].item()
    close_a = hrefs_books[key_a]
#     print(close_a)
    for c_a in close_a:
#         print()
#         print(catalog_books[catalog_books['wiki_book'] == base + c_a])
        c_a_id = catalog_books[catalog_books['wiki_book'] == base + c_a]['book_id'].item()
#         print(c_a_id)
        hrefs_ids_books[key_a_id].add(c_a_id)
        hrefs_ids_books[c_a_id].add(key_a_id)
    

In [194]:
hrefs_ids_books

defaultdict(set,
            {228: {227},
             227: {228},
             325: {322, 324},
             322: {324, 325},
             324: {322, 325},
             32: {30, 31, 34},
             30: {31, 32, 34},
             34: {30, 31, 32},
             31: {30, 32, 34},
             562: {558, 559, 560, 561},
             561: {354, 558, 559, 560, 562},
             559: {558, 560, 561, 562},
             558: {257, 559, 560, 561, 562, 564},
             560: {558, 559, 561, 562},
             568: {257, 567, 569},
             567: {568, 569},
             569: {567, 568},
             416: {418, 419},
             418: {416, 419},
             419: {315, 416, 418},
             515: {516, 517, 518, 519},
             518: {515, 516, 517, 519},
             516: {515, 517, 518, 519},
             589: {225, 591, 592},
             591: {225, 589, 592},
             592: {589, 591},
             96: {93, 98},
             93: {96, 98},
             98: {93, 96, 564},
        

In [195]:
for k in hrefs_ids_books:
    for i in catalog[catalog['book_id'] == k].index:
        catalog.at[i, 'book_com_links'] = '_'.join([str(j) for j in hrefs_ids_books[k]])

In [197]:
catalog.to_csv('collection_catalog_full_final.csv', sep=';')